Release VERSION 230109

<h1>RSoXS plan simulator</h1>
this is a full stack of the SST scan system, from the loader through to the low level acquisition code, all simulated here
run all the hidden cells first!

In [3]:
!pip install -e /nsls2/users/alevin/repos/rsoxs_scans

Obtaining file:///nsls2/users/alevin/repos/rsoxs_scans
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rsoxs-scans (pyproject.toml) ... done
  Created wheel for rsoxs-scans: filename=rsoxs_scans-0.post184+g4a1a0b6-0.editable-py3-none-any.whl size=4567 sha256=37d6826fe71742f21ed649dc08ec615e113b49d9a0db91cf2f15c94656c6b20f
  Stored in directory: /tmp/pip-ephem-wheel-cache-z49rmoyw/wheels/05/fa/82/93883113d790c18a44691574d2bc31622b743713e11acfdd31
Successfully built rsoxs-scans
  Attempting uninstall: zict
    Found existing installation: zict 3.0.0
    Uninstalling zict-3.0.0:
      Successfully uninstalled zict-3.0.0


In [1]:
%load_ext autoreload
%autoreload 2

from rsoxs_scans.spreadsheets import load_samplesxlsx, save_samplesxlsx
from rsoxs_scans.acquisition import dryrun_bar, get_acq_details

In [2]:
# test your .xlsx file here
# load the example bar
bar = load_samplesxlsx(r"Sample_Bar_v2023_1.NIST.230301Draftv2_Clean.xlsx")

spreadsheet version is 2023-1 Version 1.0
Reading sheet Acquisitions


/nsls2/users/alevin/repos/rsoxs_scans/rsoxs_scans/spreadsheets.py:354: UserWarning: Acquisition #41, sample_id:p_2 has invalid parameters: 
	Invalid Angle: 0 . For grazing samples, angle must be between 20 and 90

  warnings.warn(invalidAcqParamText)
/nsls2/users/alevin/repos/rsoxs_scans/rsoxs_scans/spreadsheets.py:431: UserWarning: Bar Sheet Entry #0, sample_id:p_2 has invalid parameters: 
	Invalid Angle: 180 . For transmission samples, angle must be between -14 and 90

  warnings.warn(invalidAcqParamText)
/nsls2/users/alevin/repos/rsoxs_scans/rsoxs_scans/spreadsheets.py:498: UserWarning: proposal 310704 is not valid for the 2023-2 cycle
 WARNING!!! no data taken with this proposal will be retrievable 
  it is HIGHLY suggested that you fix this 
 if you are running this outside of the NSLS-II network, this is expected
  warnings.warn(f"proposal {proposal} is not valid for the {cycle} cycle" + warn_text)


line 0, sample p3ht - data will not be accessible
line 1, sample pcbm - data will not be accessible


In [3]:
save_samplesxlsx(bar,'test2') # test writing to a temporary file

In [4]:
bar2 = load_samplesxlsx("test2.xlsx") # test loading from a temporary file

FileNotFoundError: [Errno 2] No such file or directory: 'test2.xlsx'

In [5]:
# dry run it
outputs = dryrun_bar(bar,['apriority'],group='all')

/nsls2/users/alevin/repos/rsoxs_scans/rsoxs_scans/acquisition.py:272: UserWarning: 
  warnings.warn(
/nsls2/users/alevin/repos/rsoxs_scans/rsoxs_scans/acquisition.py:272: UserWarning: 
  warnings.warn(
/nsls2/users/alevin/repos/rsoxs_scans/rsoxs_scans/acquisition.py:272: UserWarning: 
  warnings.warn(
/nsls2/users/alevin/repos/rsoxs_scans/rsoxs_scans/acquisition.py:272: UserWarning: 
  warnings.warn(


___________________________________________________________________
Acquisition # 0 from Group: 
	Sample Name: p3ht
	Sample id: p_1
	Project: testing

Summary: load p_1 with config SAXS, run rsoxs with priority(Sample: 1, Acquisition: 1)
	Starts @ 0:00:00 takes 0:10:00 (+120 seconds for configuration change)

> Step 0: load configuration SAXS
> Step 1: load sample p3ht
> Step 2: set Diode range to high
> Step 3: setting polarization to 0
 RSoXS scanning ['saxs_det'] from 250.0 eV to 350.0 eV on the rsoxs l/mm grating
    in 114 steps with exposure times from 1.0 to 1.0 seconds
___________________________________________________________________
___________________________________________________________________
Acquisition # 1 from Group: 
	Sample Name: pcbm
	Sample id: p_2
	Project: testing

Summary: load p_2 with config WAXS, run nexafs with priority(Sample: 2, Acquisition: 2)
	Starts @ 0:12:00 takes 0:03:10 (+120 seconds for configuration change)

> Step 0: load configuration WAXS
> 

In [6]:
# Get more details on a given acquisition
get_acq_details(40,outputs)

--------------------------------------------------
>Step: 0
--------------------------------------------------
{
    "description": "load configuration SAXS\n",
    "action": "load_configuration",
    "kwargs": {
        "configuration": "SAXS"
    },
    "queue_step": 0,
    "acq_index": 40
}
--------------------------------------------------
>Step: 1
--------------------------------------------------
{
    "description": "load sample p3ht\n",
    "action": "load_sample",
    "kwargs": {
        "sample": {
            "bar_name": "testbar",
            "sample_id": "p_1",
            "sample_name": "p3ht",
            "project_name": "testing",
            "institution": "NIST",
            "proposal_id": 310704,
            "bar_spot": "1A",
            "front": true,
            "grazing": false,
            "angle": 180,
            "height": 0.25,
            "sample_desc": "something about the sample",
            "project_desc": "I'm doing science",
            "sample_priority

In [7]:
# get the set of all actions
actions = set()
for acq in outputs:
    for step in acq['steps']:
        actions.add(step['action'])
print(actions)

{'load_sample', 'spiral_scan_core', 'load_configuration', 'rsoxs_scan_core', 'message', 'diode_high', 'diode_low', 'nexafs_scan_core'}


In [8]:
arguments = {}
for action in actions:
    keys = set()
    for acq in outputs:
        for step in acq['steps']:
            if step['action'] == action:
                if 'kwargs' in step:
                    for kwarg in step['kwargs']:
                        keys.add(kwarg)
    arguments[action]=keys
    print("{:<18} : {}".format(action,keys))

load_sample        : {'sample'}
spiral_scan_core   : {'slack_message_start', 'Parameter/ Index', 'pol', 'grating', 'exposure', 'sample_id', 'group', 'md', 'priority', 'energy', 'stepsize', 'enscan_type', 'diameter', 'dets', 'configuration', 'type', 'slack_message_end', 'uid', 'angle'}
load_configuration : {'configuration'}
rsoxs_scan_core    : {'temps_with_locations', 'slack_message_start', 'polarizations', 'Parameter/ Index', 'grating', 'sample_id', 'group', 'md', 'priority', 'locations', 'enscan_type', 'energies', 'dets', 'configuration', 'type', 'slack_message_end', 'uid', 'repeats', 'temp_wait', 'times'}
message            : set()
diode_high         : set()
diode_low          : set()
nexafs_scan_core   : {'group', 'md', 'slack_message_start', 'priority', 'configuration', 'Parameter/ Index', 'uid', 'angle', 'scan_params', 'type', 'pol', 'cycles', 'grating', 'slack_message_end', 'enscan_type', 'sample_id'}


<h1>Examples of edges, ratios, frames

In [ ]:
# totally manual simple scan
from rsoxs_scans.constructor import get_energies
energies = get_energies((1850,1930),100)

In [ ]:
# precise energy list
# you can fool the algorithm if you put 1 as the number of frames it will only take one frame at each of the thresholds, including the last one, so you can sort of treat this as just an energy list
# as a corollary, you can't have a total number of frames less than the thresholds that you give it in edges (it will at a minimum always take the threshold values)
energies = get_energies((250,250,250,340,340,341,280,281.45,500,500),1)

In [ ]:
# use a default
energies = get_energies('Nitrogen')

In [ ]:
# use a default with a different name
energies = get_energies('c')

In [ ]:
# use a different built in interval set
energies = get_energies('c','full','carbon nonaromatic')

In [ ]:
# get a very short version of this scan
energies = get_energies('c','very short','carbon nonaromatic')

In [ ]:
# I don't have time for this, I only want ~10 energies
energies = get_energies('c',10,'carbon nonaromatic')

In [ ]:
# also, I made single entry energies also work
energies = get_energies(280)

In [ ]:
# also option to have frames as a list of exact numbers needed in each region
energies = get_energies('c',[2,5,10,40,5,2])

In [ ]:
# putting in zero effectively removes that region
energies = get_energies('c',[2,5,10,0,5,2])

In [ ]:
from rsoxs_scans.constructor import construct_exposure_times
times,_ = construct_exposure_times(get_energies([270,280,290,400],500,[2,1,10]),1)
len(times)

In [ ]:
from rsoxs_scans.constructor import get_nexafs_scan_params
get_nexafs_scan_params('n','normal')

<h1>Example of complex Exposure times

In [ ]:
construct_exposure_times(get_energies((1850,1930),100),[2,("between",1870,1900.0),4,("greater_than",1920.0),1],80)

In [ ]:
isinstance('asdf',str)